# ISIS 4822 - Laboratorio 2

Hernán Cuy - 202010199

Camilo Rozo - 201820147

---

## Preprocesamiento de datos

---

### Load dataset

In [17]:
import pandas as pd
import numpy as np
from pathlib import Path
import json


def converter(obj):
    if isinstance(obj, np.integer | np.int64 | np.int32 | np.int16 | np.int8):
        return int(obj)
    elif isinstance(obj, np.floating | np.float64 | np.float32 | np.float16):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.str_):
        return str(obj)
    elif isinstance(obj, np.void):
        return None
    return obj

terridata_filename = "TerriData_Dim4.csv"

data_path = Path('data')

sources_path = data_path.joinpath("sources")

terridata_path = sources_path.joinpath(terridata_filename)

terridata = pd.read_csv(terridata_path,encoding='utf-8')

### Typify dataset to JSON-valid values

In [18]:

def transform_datatypes(df: pd.DataFrame) -> pd.DataFrame:
    df["Departamento"]=df["Departamento"].astype('str') 
    df["Entidad"]=df["Entidad"].astype('str')
    df["Dimensión"]=df["Dimensión"].astype('str')
    df["Subcategoría"]=df["Subcategoría"].astype('str') 
    df["Indicador"]=df["Indicador"].astype('str')
    df["Dato Numérico"] = df["Dato Numérico"].apply(lambda x: float(str(x).replace(",", ".")) if x is not None else None)
    df["Dato Cualitativo"]=df["Dato Cualitativo"].astype('str')
    df["Fuente"]=df["Fuente"].astype('str')
    df["Unidad de Medida"]=df["Unidad de Medida"].astype('str') 
    return df

terridata = transform_datatypes(terridata)

### View resulting dataset

In [19]:
terridata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291756 entries, 0 to 291755
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Código Departamento  291756 non-null  int64  
 1   Departamento         291756 non-null  object 
 2   Código Entidad       291756 non-null  int64  
 3   Entidad              291756 non-null  object 
 4   Dimensión            291756 non-null  object 
 5   Subcategoría         291756 non-null  object 
 6   Indicador            291756 non-null  object 
 7   Dato Numérico        269714 non-null  float64
 8   Dato Cualitativo     291756 non-null  object 
 9   Año                  291756 non-null  int64  
 10  Mes                  291756 non-null  int64  
 11  Fuente               291756 non-null  object 
 12  Unidad de Medida     291756 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 28.9+ MB


In [20]:
terridata.head()

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Educación,Acceso a la educación,Cobertura bruta en transición,95.42,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5000,Antioquia,Educación,Acceso a la educación,Cobertura bruta en transición,78.62,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
2,5,Antioquia,5001,Medellín,Educación,Acceso a la educación,Cobertura bruta en transición,103.23,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
3,5,Antioquia,5002,Abejorral,Educación,Acceso a la educación,Cobertura bruta en transición,51.80,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
4,5,Antioquia,5004,Abriaquí,Educación,Acceso a la educación,Cobertura bruta en transición,96.00,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


### Process the dataset according to the tasks

_"El Ministerio de Educación quiere utilizar estos datos para verificar la evolución de la calidad de la educación primaria y secundaria en el país en los últimos 12 años e identificar las regiones geográficas con mayor cobertura e incidencia de la educación en estos mismos periodos."_

Tareas identificadas:

1. verificar la evolución de la calidad de la educación primaria y secundaria en el país en los últimos 12 años.
2. identificar las regiones geográficas con mayor cobertura e incidencia de la educación en estos mismos periodos.

|Observación|Acción|Justificación|
|:---|:---|:---|
|Período de los últimos 12 años|**Eliminar** años previos a **2010** |La información no será usada para ninguna tarea |
|Un objetivo es ver cobertura|**Conservar** indicadores relacionados a la subcategoría **"Acceso a la educación"**|Muy probablemente la visualización usará estos datos|
|Ambos objetivos hacen énfasis en geografía |**Dividir** el dataset por **territorio** en lugar de subcategorías e indicadores |Visualizar información a nivel país y a nivel regional requiere segregación territorial|
|La temporalidad es anual|**Eliminar Mes**|no es necesario|
|El dataset tiene una única dimensión|**Eliminar Dimensión**| no es necesaria por tener un único valor|
|Las métricas son numéricas|**Eliminar Dato Cualitativo**|no es necesaria por tener un único valor|
|Los indicadores dan toda la información necesaria sobre el dato|**Eliminar Subcategoría**|  sólo cumple un rol semántico|
|Las tareas requieren división demográfica por región y tipo de educación|**Eliminar Acceso a la educación desagregado por sexo, Logro**|El objetivo no es segregar por edad ni sexo|

Subcategorías **si** relacionadas con la tarea 1.
  
  - 

Subcategorías **no** relacionadas con la tarea 1.

 - Acceso a la educación desagregado por sexo, Acceso a la educación


Eliminar años previos a 2010

In [21]:
terridata = terridata[terridata["Año"] >= 2010]

Eliminar filas y columnas innecesarias

In [22]:
del terridata["Dimensión"]
del terridata["Mes"]
del terridata["Dato Cualitativo"]
del terridata["Subcategoría"]

Hallar datos departamentales

In [42]:
data_departamental = terridata[(terridata["Departamento"] == terridata["Entidad"]) & (terridata["Código Entidad"] % 1000 == 0)]
# la condición del AND es porque existe sucre, sucre, que es municipio (cod 70771)

departamentos = data_departamental["Código Departamento"].drop_duplicates(
).to_list()

anios = terridata["Año"].drop_duplicates().to_list()

deptos_map = {}

for depto in departamentos:
    row = data_departamental[data_departamental["Código Departamento"] == depto].iloc[0]
    
    deptos_map[depto] = {
        "nombre": row["Departamento"],
        "codigo": row["Código Departamento"],
        "codigo_entidad": row["Código Entidad"],

    }


Hallar datos municipales

In [24]:
data_municipal = terridata[(terridata["Departamento"] != terridata["Entidad"]) | (terridata["Código Entidad"] % 1000 != 0)]

municipios = data_municipal["Código Entidad"].to_list()

municipios_map = {}

for municipio in municipios:
    row = data_municipal[data_municipal["Código Entidad"] == municipio].iloc[0]
    
    municipios_map[municipio] = {
        "nombre": row["Entidad"],
        "codigo": row["Código Entidad"],
        "codigo_departamento": row["Código Departamento"],
        "nombre_departamento": row["Departamento"]
    }


KeyboardInterrupt: 

In [ ]:
json_departamental = pd.read_json('data_departamental.json',encoding='utf-8')
json_departamental = transform_datatypes(json_departamental)
json_departamental.head()

In [ ]:
indicadores = ["Cobertura bruta en educación - Total",
               "Cobertura neta en educación - Total"]

datos_cobertura_bruta_neta = json_departamental.loc[np.where(
    json_departamental['Indicador'].isin(indicadores))]

keys_to_keep = ["Código Departamento", "Departamento",
                "Indicador", "Año", "Dato Numérico", "Unidad de Medida"]

for year in list(datos_cobertura_bruta_neta.keys()):
    if year not in keys_to_keep:
        del datos_cobertura_bruta_neta[year]

differenceMap = {anio: {row["Código Departamento"]: {
    "Departamento": row["Departamento"],
    "Año": row["Año"],
    "Dato Numérico": row["Dato Numérico"],
    "Unidad de Medida": row["Unidad de Medida"]

} for row in datos_cobertura_bruta_neta.to_dict(orient="index", into=dict).values()} for anio in anios}


for row in datos_cobertura_bruta_neta.to_dict(orient="index", into=dict).values():
    if row["Indicador"] == "Cobertura bruta en educación - Total":
        differenceMap[row["Año"]][row["Código Departamento"]
                                  ]["Cobertura bruta en educación - Total"] = row["Dato Numérico"]
    else:
        differenceMap[row["Año"]][row["Código Departamento"]
                                  ]["Cobertura neta en educación - Total"] = row["Dato Numérico"]

min_diff = float("inf")
max_diff = float("-inf")

for year in differenceMap:
    for department in differenceMap[year]:
        resta = differenceMap[year][department]["Cobertura bruta en educación - Total"] - differenceMap[year][department]["Cobertura neta en educación - Total"]
        if resta < min_diff:
            min_diff = resta
        if resta > max_diff:
            max_diff = resta
        differenceMap[year][department]["Diferencia"] =  resta 

datos_cobertura_bruta_neta = differenceMap



In [ ]:



convertedTypes = {converter(year): {converter(department): {key: converter(value) for key, value in datos_cobertura_bruta_neta[year][department].items(
)} for department in datos_cobertura_bruta_neta[year]} for year in datos_cobertura_bruta_neta}

convertedTypes["min"] = min_diff
convertedTypes["max"] = max_diff


### Tarea 1 - Calidad de educación primaria y secundaria nacional a lo largo del tiempo



vis1 - linechart

item: promedio nacional de puntaje saber 11 en un área determinada (derivado) 

marca: 
 - puntos unidos con líneas

canales: 
 - posición horizontal (año)
 - posición vertical (puntaje promedio)
 - color (área de la prueba)


In [52]:
avgs = {}

keys = ["Puntaje promedio Pruebas Saber 11 - Matemáticas",
        "Puntaje promedio Pruebas Saber 11 - Lectura crítica"]

jsonobject = {"data": avgs, "min": float("inf"), "max": float("-inf")}
for key in keys: 
  keyavgs = []
  for anio in anios:
      obj = {"year": anio,
          "value": round(data_departamental.iloc[np.where((data_departamental["Año"] == anio) & (data_departamental["Indicador"] == key))]["Dato Numérico"].mean(),3)
        }
      if obj["value"] < jsonobject["min"]:
          jsonobject["min"] = obj["value"]
      if obj["value"] > jsonobject["max"]:
          jsonobject["max"] = obj["value"]
      keyavgs.append(obj)
  avgs[key] = keyavgs

path = "data/nacional/nacional_acum_saber11.json"

with open(path, 'w',encoding="utf-8") as outfile:
    json.dump(jsonobject, outfile,indent=4, ensure_ascii=False)


### Tarea 2 - Cobertura de educación por región a lo largo del tiempo

vis2 - choropleth

idea: mostrar diferencia de cobertura bruta (porque solo es cobertura sin calidad) en educacion total (abarcar todas de una) a lo largo del tiempo

item: 
 - 
marca: 
 - puntos unidos con líneas

canales: 
 - posición horizontal (año)
 - posición vertical (puntaje promedio)
 - color (área de la prueba)